In [3]:
from dotenv import load_dotenv
import os, sys
from pathlib import Path

load_dotenv()   
src_path = Path(os.getcwd()).resolve().parents[1]   
sys.path.append(str(src_path))  

In [ ]:
# %%writefile fairy_guild_agent.py
from prompts.promptmanager import PromptManager
from prompts.prompt_type.fairy.FairyPromptType import FairyPromptType
from langchain_core.messages import SystemMessage
from agents.fairy.util import get_groq_llm_lc
from langgraph.graph import START, END, StateGraph
from agents.fairy.fairy_state import FairyGuildState

from langchain.chat_models import init_chat_model
from enums.LLM import LLM
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from agents.fairy.util import find_scenarios, str_to_bool, find_heroine_info
from agents.fairy.cache_data import GAME_SYSTEM_INFO


fast_llm = init_chat_model(
    model=LLM.GROK_4_FAST_NON_REASONING, 
    model_provider="xai"
)

reasoning_llm = init_chat_model(
    model=LLM.GROK_4_FAST_REASONING, 
    model_provider="xai", 
    temperature=0.2
)

@tool
def get_scenarios(config: RunnableConfig):
    """히로인의 과거 데이터 입니다. 히로인 정보에 있는 히로인 일때만 데이터를 찾습니다."""
    heroine_id = config.get("configurable", {}).get("heroine_id")
    memory_progress = config.get("configurable", {}).get("memory_progress")
    return f"""[히로인의 과거]\n{find_scenarios(heroine_id, memory_progress)}"""


@tool
def get_game_system():
    """사용자가 현재 게임의 시스템적 기능, 구조, 진행 방식, 옵션, 조작 방식 등에 대해 질문할 때 선택됩니다."""
    return GAME_SYSTEM_INFO

@tool
def get_heroine_info(config: RunnableConfig):
    """(루파메스,레티아,로코 중 1명) 히로인의 현재 상태에 대한 데이터 입니다."""
    memory_progress = config.get("configurable", {}).get("memory_progress")
    affection = config.get("configurable", {}).get("affection")
    sanity = config.get("configurable", {}).get("sanity")
    heroine_data = {
        "affection": affection,
        "memoryProcess": memory_progress,
        "sanity": sanity,
    }
    return f"""
    ## 데이터 설명
    | 수치 | 범위 | 설명 |
    | ----- | ----- | ----- |
    | **affection** | 0\~100 | 호감도. 대화/행동으로 증감 |
    | **memoryProgress** | 0\~100 | 기억진척도. 해금 단계별로 갱신 |
    | **sanity** | 0\~100 | 정신 안정도. 0이면 우울 상태 |

    ## 현재 히로인의 데이터(JSON)
    {heroine_data}
    """


def reasoning_required(state: FairyGuildState):
    last_meesage = state["messages"][-1].content
    reasoning_required_prompt = PromptManager(
        FairyPromptType.FAIRY_REASONING_REQUIRED
    ).get_prompt(question=last_meesage)
    return {
        "reasoning_required": str_to_bool(
            get_groq_llm_lc(LLM.LLAMA_3_1_8B_INSTANT)
            .invoke(reasoning_required_prompt)
            .content
        )
    }


def call_llm(state: FairyGuildState, config: RunnableConfig):
    heroine_id = config.get("configurable", {}).get("heroine_id")
    heroine_info = find_heroine_info(heroine_id=heroine_id)

    messages = state["messages"]

    if state["reasoning_required"]:
        llm = reasoning_llm.bind_tools([get_scenarios, get_game_system, get_heroine_info])
    else:
        llm = fast_llm

    system_prompt = PromptManager(FairyPromptType.FAIRY_GUILD_SYSTEM).get_prompt(
        heroine_info=heroine_info
    )
    new_messages = [SystemMessage(content=system_prompt)] + messages
    ai_answer = llm.invoke(new_messages)
    return {"messages": [ai_answer]}


graph_builder = StateGraph(FairyGuildState)

graph_builder.add_node("reasoning_required", reasoning_required)
graph_builder.add_node("call_llm", call_llm)
tool_node = ToolNode([get_scenarios, get_game_system, get_heroine_info])
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "reasoning_required")
graph_builder.add_edge("reasoning_required", "call_llm")
graph_builder.add_edge("call_llm", END)

graph_builder.add_conditional_edges("call_llm", tools_condition)
graph_builder.add_edge("tools", "call_llm")
graph_builder

Overwriting fairy_guild_agent.py


In [ ]:
from agents.fairy.fairy_guild_agent import graph_builder as guild_builder
from agents.fairy.util import add_human_message
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = guild_builder.compile(checkpointer=memory)
config = {
    "configurable": {"heroine_id": 1, "memory_progress": 90, "affection":50, "sanity": 50, "thread_id": "player-900"}
}

graph.invoke(
    {"messages": [add_human_message("히로인 과거에 대해 알려줘")]},
    config=config,
)

/Users/seobi/PythonProjects/ProjectML/src/agents/fairy/fairy_guild_agent.py:61: SyntaxWarning: invalid escape sequence '\~'
  """
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


{'messages': [HumanMessage(content='히로인 과거에 대해 알려줘', additional_kwargs={'created_at': '2025-12-03T20:19:23.169069'}, response_metadata={}, id='7ccd7e32-012f-4ddf-91ab-cf14286aff9f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 3635, 'total_tokens': 4091, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 440, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 305, 'text_tokens': 3635, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_adbb3573fd', 'id': 'b978e7f3-6993-9d4c-2f26-ff781acb2076', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2a501c84-97cb-4935-a2ef-198451627146-0', tool_calls=[{'name': 'get_scenarios', 'args': {}, 'id': 'call_16093945', 'type': 'tool_call'}], usage_metadata={'input_tokens'

In [4]:
graph.invoke(
    {"messages": [add_human_message("너무 슬프잖아..")]},
    config=config,
)

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


{'messages': [HumanMessage(content='히로인 과거에 대해 알려줘', additional_kwargs={'created_at': '2025-12-03T20:19:23.169069'}, response_metadata={}, id='7ccd7e32-012f-4ddf-91ab-cf14286aff9f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 3635, 'total_tokens': 4091, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 440, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 305, 'text_tokens': 3635, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_adbb3573fd', 'id': 'b978e7f3-6993-9d4c-2f26-ff781acb2076', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2a501c84-97cb-4935-a2ef-198451627146-0', tool_calls=[{'name': 'get_scenarios', 'args': {}, 'id': 'call_16093945', 'type': 'tool_call'}], usage_metadata={'input_tokens'

In [5]:
graph.invoke(
    {"messages": [add_human_message("로코에 대해서 알려줘..")]},
    config=config,
)

{'messages': [HumanMessage(content='히로인 과거에 대해 알려줘', additional_kwargs={'created_at': '2025-12-03T20:19:23.169069'}, response_metadata={}, id='7ccd7e32-012f-4ddf-91ab-cf14286aff9f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 3635, 'total_tokens': 4091, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 440, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 305, 'text_tokens': 3635, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_adbb3573fd', 'id': 'b978e7f3-6993-9d4c-2f26-ff781acb2076', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2a501c84-97cb-4935-a2ef-198451627146-0', tool_calls=[{'name': 'get_scenarios', 'args': {}, 'id': 'call_16093945', 'type': 'tool_call'}], usage_metadata={'input_tokens'

In [6]:
graph.invoke(
    {"messages": [add_human_message("간단한 조작키좀 알려줘")]},
    config=config,
)

{'messages': [HumanMessage(content='히로인 과거에 대해 알려줘', additional_kwargs={'created_at': '2025-12-03T20:19:23.169069'}, response_metadata={}, id='7ccd7e32-012f-4ddf-91ab-cf14286aff9f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 3635, 'total_tokens': 4091, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 440, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 305, 'text_tokens': 3635, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_adbb3573fd', 'id': 'b978e7f3-6993-9d4c-2f26-ff781acb2076', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2a501c84-97cb-4935-a2ef-198451627146-0', tool_calls=[{'name': 'get_scenarios', 'args': {}, 'id': 'call_16093945', 'type': 'tool_call'}], usage_metadata={'input_tokens'

In [9]:
graph.invoke(
    {"messages": [add_human_message("내 히로인에게 호감을 얻기 위해서는 어떻게 해야해?")]},
    config=config,
)

{'messages': [HumanMessage(content='히로인 과거에 대해 알려줘', additional_kwargs={'created_at': '2025-12-03T20:19:23.169069'}, response_metadata={}, id='7ccd7e32-012f-4ddf-91ab-cf14286aff9f'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 3635, 'total_tokens': 4091, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 440, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 305, 'text_tokens': 3635, 'image_tokens': 0}, 'num_sources_used': 0}, 'model_provider': 'xai', 'model_name': 'grok-4-fast-reasoning', 'system_fingerprint': 'fp_adbb3573fd', 'id': 'b978e7f3-6993-9d4c-2f26-ff781acb2076', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--2a501c84-97cb-4935-a2ef-198451627146-0', tool_calls=[{'name': 'get_scenarios', 'args': {}, 'id': 'call_16093945', 'type': 'tool_call'}], usage_metadata={'input_tokens'

In [7]:
from core.game_dto.z_muck_factory import MockFactory
MockFactory.create_dungeon_player(1).model_dump_json()

'{"playerId":1,"affection":10,"heroineId":0,"sanity":2,"scenarioLevel":5,"difficulty":0,"stats":{"hp":476,"moveSpeed":1.41,"cooldownReduction":1.72,"strength":15,"dexterity":18,"intelligence":null,"critChance":35.18,"skillDamageMultiplier":1.5,"autoAttackMultiplier":1.98,"attackSpeed":1.44},"skills":{"passiveSkillId":4,"passiveSkillLevel":3,"activeSkillId":4,"activeSkillLevel":2},"weapon":{"weaponId":null,"weaponType":2,"weaponName":null,"rarity":0,"attackPower":39,"staggerPower":null,"modifier":{"strength":0.5,"dexterity":0.3}},"inventory":[0,21,42]}'